## ランダムフォレスト　デフォルト

In [1]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn import tree


C:\Users\owner\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [2]:
# pickle ファイルから読み込み
train_pkl = pd.read_pickle('./train.pk1')

In [4]:
train_pkl.shape

(55175, 12)

In [5]:
train_pkl.describe()

,id,accommodates,bathrooms,bedrooms,beds,number_of_reviews,review_scores_rating,y
count,55175.000000,55175.000000,55175.000000,55175.000000,55175.000000,55175.000000,55175.000000,55175.000000
mean,27792.028944,3.158496,1.237191,1.266298,1.713983,20.933158,94.520616,160.366706
std,16044.979801,2.155461,0.584007,0.850715,1.259778,37.964139,6.906614,168.356651
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,20.000000,1.000000
25%,13902.500000,2.000000,1.000000,1.000000,1.000000,1.000000,93.000000,75.000000
50%,27784.000000,2.000000,1.000000,1.000000,1.000000,6.000000,96.000000,112.000000
75%,41687.500000,4.000000,1.000000,1.000000,2.000000,23.000000,99.000000,185.000000
max,55582.000000,16.000000,8.000000,10.000000,18.000000,605.000000,100.000000,1999.000000


## 訓練データとテストデータに分割

In [6]:
# 訓練とテストデータに分割
train, test = train_test_split(train_pkl, test_size=0.2, random_state=42)

In [7]:
# ターゲットと特徴量の分割
train_X = train.iloc[:, 1:-1].values
train_y = train.y.values

## 訓練データで訓練

In [8]:
RF = RandomForestRegressor(random_state=42)

In [9]:
RF = RF.fit(train_X, train_y)

In [10]:
RF.feature_importances_

array([0.1274006 , 0.20543972, 0.31261551, 0.06899632, 0.02959755,
       0.00350868, 0.03499634, 0.0286666 , 0.13427776, 0.05450091])

In [11]:
sorted(
    zip(map(lambda x: round(x, 3), RF.feature_importances_), train.iloc[:, 1:].columns),
    reverse=True)

[(0.313, 'bedrooms'),
 (0.205, 'bathrooms'),
 (0.134, 'number_of_reviews'),
 (0.127, 'accommodates'),
 (0.069, 'beds'),
 (0.055, 'review_scores_rating'),
 (0.035, 'host_identity_verified'),
 (0.03, 'cleaning_fee'),
 (0.029, 'instant_bookable'),
 (0.004, 'host_has_profile_pic')]

In [22]:
# 訓練済みの決定木を視覚化
# dot_data = tree.export_graphviz(RF, out_file=None,
#                                feature_names=train.iloc[:, 1:].columns,
#                                class_names=train.Survived.name,
#                                rounded=True,
#                                filled=True,
#                                special_characters=True)

## テストデータで実行

In [15]:
# ターゲットと特徴量の分割
test_x = test.iloc[:, 1:-1].values
test_y = test.y.values

In [16]:
test_x.shape, test_y.shape

((11035, 10), (11035,))

In [17]:
pred_y = RF.predict(test_x)

In [19]:
np.sqrt(mean_squared_error(test_y, pred_y))

137.76944239575494

## 検証データで実行

In [20]:
# 検証データ読み込み
valid = pd.read_pickle('./test.pk1')

In [21]:
valid.shape

(18528, 11)

In [22]:
# ID の保存
valid_pass = valid.id.values

In [23]:
valid_X = valid.iloc[:, 1:]

In [24]:
valid_X.describe()

,accommodates,bathrooms,bedrooms,beds,host_has_profile_pic,host_identity_verified,number_of_reviews,review_scores_rating
count,18528.000000,18528.000000,18528.000000,18528.000000,18528.000000,18528.000000,18528.000000,18528.000000
mean,3.161863,1.231110,1.266947,1.706822,0.003022,0.324050,20.790425,94.477386
std,2.155397,0.578248,0.860160,1.242269,0.054895,0.468031,37.457273,7.037526
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000
25%,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,93.000000
50%,2.000000,1.000000,1.000000,1.000000,0.000000,0.000000,6.000000,96.000000
75%,4.000000,1.000000,1.000000,2.000000,0.000000,1.000000,23.000000,99.000000
max,16.000000,8.000000,10.000000,16.000000,1.000000,1.000000,451.000000,100.000000


In [25]:
valid_X.shape, train_X.shape

((18528, 10), (44140, 10))

In [26]:
pred_valid_y = RF.predict(valid_X)

In [27]:
pred_valid_y.shape

(18528,)

In [28]:
type(valid_pass), type(pred_valid_y)

(numpy.ndarray, numpy.ndarray)

In [29]:
result_df = pd.DataFrame(pred_valid_y, valid_pass, columns=['y'])

In [30]:
result_df.head()

,y
0,154.970000
1,160.965761
2,200.370222
3,207.329140
4,103.117286


In [31]:
result_df.to_csv("./RF_1.csv", header=False)